In [1]:
# load env
import dotenv
dotenv.load_dotenv()

True

In [2]:
virtual_app = dict(
    llm=dict(
        modelname="some llm component model name"
    ),
    template="information about the template I used in my app",
    debug="all of these fields are completely optional"
)
from trulens_eval import Select
from trulens_eval.tru_virtual import VirtualApp

virtual_app = VirtualApp(virtual_app) # can start with the prior dictionary
virtual_app[Select.RecordCalls.llm.maxtokens] = 1024

Package llama-index is installed but has a version conflict:
	(llama-index 0.9.21 (/Users/noble/opt/anaconda3/envs/mayil_ai/lib/python3.10/site-packages), Requirement.parse('llama-index>=0.9.26'))

This package is optional for trulens_eval so this may not be a problem but if
you need to use the related optional features and find there are errors, you
will need to resolve the conflict:

    ```bash
    pip install 'llama-index>=0.9.26'
    ```

If you are running trulens_eval in a notebook, you may need to restart the
kernel after resolving the conflict. If your distribution is in a bad place
beyond this package, you may need to reinstall trulens_eval so that all of the
dependencies get installed and hopefully corrected:
    
    ```bash
    pip uninstall -y trulens_eval
    pip install trulens_eval
    ```

Package chromadb is installed but has a version conflict:
	(chromadb 0.4.14 (/Users/noble/opt/anaconda3/envs/mayil_ai/lib/python3.10/site-packages), Requirement.parse('chromadb>=0.4

In [3]:
from trulens_eval import Select
retriever = Select.RecordCalls.retriever
synthesizer = Select.RecordCalls.synthesizer

virtual_app[retriever] = "retriever"
virtual_app[synthesizer] = "synthesizer"

In [4]:
from trulens_eval.tru_virtual import VirtualRecord

# The selector for a presumed context retrieval component's call to
# `get_context`. The names are arbitrary but may be useful for readability on
# your end.
context_call = retriever.get_context
generation = synthesizer.generate

import json

with open("./data/test.json", "r") as f:
    ground_truth = json.load(f)

data = []
task_instances = []
for issue_details in ground_truth:
    with open(f"./data/v1/{issue_details['instance_id']}.json", "r") as f:
        generated_details = json.load(f)
    problem_statement = issue_details["problem_statement"]
    mayil_response = generated_details["mayil_response"]
    rets = [f"Filename: {c['filename']} | (Lines: {c['start_line']} to {c['end_line']})\nCode Snippet:\n{c['code']}" for c in generated_details["mayil_collected_data"]["relevant_snippets"]]
    if not mayil_response:
        continue

    rec = VirtualRecord(
        main_input=problem_statement,
        main_output=mayil_response,
        calls=
            {
                context_call: dict(
                    args=[problem_statement],
                    rets=rets
                ),
                generation: dict(
                    args=["""
                        We have provided the below context: \n
                        ---------------------\n
                        """+ "\n".join(rets) +
                        """
                        ---------------------\n
                        Given this information, please create a response to guide the developer in solving the issue:
                        """+ problem_statement],
                    rets=[mayil_response]
                )
            }
        )
    data.append(rec)

In [5]:
len(data)

2280

In [6]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.feedback.feedback import Feedback

# Initialize provider class
openai = OpenAI()

# Select context to be used in feedback. We select the return values of the
# virtual `get_context` call in the virtual `retriever` component. Names are
# arbitrary except for `rets`.
context = context_call.rets[:]

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(openai.qs_relevance)
    .on_input()
    .on(context)
)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=openai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on_input_output()
)

✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input context will be set to __record__.app.retriever.get_context.rets[:] .
✅ In Groundedness, input source will be set to __record__.app.retriever.get_context.rets[:].collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [7]:
from trulens_eval.tru_virtual import TruVirtual

virtual_recorder = TruVirtual(
    app_id="Mayil",
    app=virtual_app,
    feedbacks=[f_context_relevance, f_groundedness, f_qa_relevance],
    feedback_mode = "deferred" # optional
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [8]:
for record in data:
    virtual_recorder.add_record(record)

In [9]:
from trulens_eval import Tru
tru = Tru()

# tru.run_dashboard(force=True)

In [ ]:
tru.start_evaluator()

# tru.stop_evaluator() # stop if needed

Will keep max of 32 feedback(s) running.
Tasks are spread among max of 128 thread(s).
Will rerun running feedbacks after a minute.
Will rerun failed feedbacks after 5 minutes.


<Thread(Thread-4 (_future_target_wrapper), started daemon 11688505344)>

Feedback Status:   0%|          | 0/6840 [00:00<?, ?feedbacks/s, NONE=6840]

Done Runs: 0runs [00:00, ?runs/s]

Waiting for Runs: 0runs [00:00, ?runs/s]

...

Groundedness per statement in source:   0%|          | 0/11 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/10 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/17 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/10 [00:00<?, ?it/s]

openai request failed <class 'openai.RateLimitError'>=Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Fr9UDIYK44Xr4pHLhECIGN9O on tokens per min (TPM): Limit 60000, Used 59733, Requested 2849. Please try again in 2.582s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Retries remaining=3.
openai request failed <class 'openai.InternalServerError'>=Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_1bf53162f63329d946b49f029af40238 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}. Retries remaining=3.
openai request failed <class 'openai.RateLimitError'>=Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-